In [10]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

In [11]:
HF_TOKEN = os.getenv("HF_TOKEN")
HF_TOKEN.startswith("hf_")

True

## Dataset Preprocessing

In [12]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Using the latest cached version of the dataset since glue couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'mrpc' at /Users/thomas/.cache/huggingface/datasets/glue/mrpc/0.0.0/bcdcba79d07bc864c1c254ccfcedcce55bcc9a8c (last modified on Sun Mar  2 20:51:21 2025).


## Training

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments("mrpc-output") # this is the output directory

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/var/folders/vc/tgvh_sd575vgbr2w_c34bh640000gp/T/ipykernel_78575/2700434406.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()

Step,Training Loss
500,0.556500
1000,0.343000


TrainOutput(global_step=1377, training_loss=0.38790354455132026, metrics={'train_runtime': 501.5406, 'train_samples_per_second': 21.94, 'train_steps_per_second': 2.746, 'total_flos': 405114969714960.0, 'train_loss': 0.38790354455132026, 'epoch': 3.0})